In [ ]:
!pip install yfinance
%pip install openai==1.12.0



In [68]:
from notebookutils.mssparkutils.credentials import getSecret

keyvault = "https://finanaceaiKeyvault.vault.azure.net/"

openaiapi = getSecret(keyvault, "openai-api")

StatementMeta(, 24e78df5-843c-459e-9ee2-e66af4d91ce9, 88, Finished, Available)

In [69]:
import openai
from openai import AzureOpenAI
import pandas as pd
import yfinance as yf

data_list = []

client = AzureOpenAI(
    azure_endpoint="https://polite-ground-030dc3103.4.azurestaticapps.net/api/v1",
    api_key=openaiapi,
    api_version="2023-09-01-preview",
)

# Function to generate a concise summary using Azure OpenAI
def generate_concise_summary(info):
    messages = [
        {"role": "system", "content": "you are a summarizer bot"},
        {"role": "user", "content": "Summarize the information below, make it more concise, not more than 200 characters"},
        {"role": "user", "content": info}
    ]
    completion = client.chat.completions.create(model="gpt-35-turbo", messages=messages, temperature=0.9)
    if completion.choices:
        return completion.choices[0].message.content
    else:
        return "Summary generation failed"

ticker_symbols = 'MSFT AAPL 2222.SR GOOGL AMZN NVDA META BRK-A LLY TSM PEP SAP TMO LIN 601857.SS ROG.SW NVS MCD ABT DIS'
tickers = yf.Tickers(ticker_symbols)

summary_df = pd.DataFrame(columns=['Ticker', 'Concise Summary'])

# Iterate over each ticker
for ticker in ticker_symbols.split():
    info = tickers.tickers[ticker].info.get('longBusinessSummary', 'No information available')
    
    # Generate a concise summary for each ticker
    concise_summary = generate_concise_summary(info)  # Placeholder for actual summarization logic
    
    # Append the ticker and its concise summary to the DataFrame
    data_list.append({'Ticker': ticker, 'Concise Summary': concise_summary})



combined_data = pd.DataFrame(data_list)

combined_data

StatementMeta(, 24e78df5-843c-459e-9ee2-e66af4d91ce9, 89, Finished, Available)

,Ticker,Concise Summary
0,MSFT,"Microsoft Corporation develops software, servi..."
1,AAPL,"Apple Inc. designs and sells smartphones, comp..."
2,2222.SR,Saudi Arabian Oil Company is an integrated ene...
3,GOOGL,Alphabet Inc. is a multinational corporation t...
4,AMZN,Amazon.com is a retail company that operates o...
5,NVDA,"NVIDIA provides graphics, networking, and comp..."
6,META,"Meta Platforms, Inc. develops products for con..."
7,BRK-A,Berkshire Hathaway is a multinational conglome...
8,LLY,Eli Lilly develops and markets pharmaceuticals...
9,TSM,Taiwan Semiconductor Manufacturing Company (TS...


In [73]:
# Convert the combined pandas DataFrame to a Spark DataFrame
combined_spark_df = spark.createDataFrame(combined_data)

# Show the Spark DataFrame to verify the structure
combined_spark_df.show()

StatementMeta(, 24e78df5-843c-459e-9ee2-e66af4d91ce9, 93, Finished, Available)

+---------+--------------------+
|   Ticker|     Concise Summary|
+---------+--------------------+
|     MSFT|Microsoft Corpora...|
|     AAPL|Apple Inc. design...|
|  2222.SR|Saudi Arabian Oil...|
|    GOOGL|Alphabet Inc. is ...|
|     AMZN|Amazon.com is a r...|
|     NVDA|NVIDIA provides g...|
|     META|Meta Platforms, I...|
|    BRK-A|Berkshire Hathawa...|
|      LLY|Eli Lilly develop...|
|      TSM|Taiwan Semiconduc...|
|      PEP|PepsiCo manufactu...|
|      SAP|SAP SE provides g...|
|      TMO|Thermo Fisher Sci...|
|      LIN|Linde plc is a gl...|
|601857.SS|PetroChina engage...|
|   ROG.SW|Roche Holding AG ...|
|      NVS|Novartis AG is a ...|
|      MCD|McDonald's is a m...|
|      ABT|Abbott Laboratori...|
|      DIS|Walt Disney Compa...|
+---------+--------------------+



In [80]:
from pyspark.sql.functions import col

combined_spark_df = combined_spark_df.select([col(c).alias(
        c.replace( '(', '')
        .replace( ')', '')
        .replace( ',', '')
        .replace( ';', '')
        .replace( '{', '')
        .replace( '}', '')
        .replace( '\n', '')
        .replace( '\t', '')
        .replace( ' ', '_')
    ) for c in combined_spark_df.columns])

StatementMeta(, 24e78df5-843c-459e-9ee2-e66af4d91ce9, 100, Finished, Available)

In [81]:
combined_spark_df.write.mode("overwrite").format("delta").save("Tables/" + 'CompanySummary')

StatementMeta(, 24e78df5-843c-459e-9ee2-e66af4d91ce9, 101, Finished, Available)